# Introduction
This tutorial provides a minimal working example demonstrating how to use the NanoVer-IMD visualisation package in conjunction with the parser package to render results derived from the analysis of Dynamical Non-Equilibrium Molecular Dynamics (D-NEMD) simulations. The walkthrough covers loading the data, configuring visualisation settings, and running the visualiser to display the displacement data.

# Setup
Before proceeding, it is essential to ensure that all the required software is installed and correctly configured. The D-NEMD visualiser script necessitates a special feature branch of NanoVer-IMD to function properly. This specific version can be downloaded **[here](https://github.com/mcsloy/NanoVerNemdParser/releases/tag/Pre-release)**. Given that this script will not operate with any other branch of NanoVer-IMD it is the user's responsibility to verify that they are using the correct version of this software. No special installation instructions are necessary, as the binary executable can be downloaded and run directly.

An up-to-date version of the NanoVer-protocol Python package is also required. It can be installed via Anaconda by issuing the command

```sh
conda install -c irl -c conda-forge nanover-server
``` 

Finally, the NanoVer D-NEMD parser package contained within this repository must be download and installed like so:
```sh
git clone git@github.com:mcsloy/NanoVerNemdParser.git && cd NanoVerNemdParser
pip install -e .
```
Finally, the following code block must be run in order to import all the relevant python packages.

In [ ]:
from MDAnalysis import Universe
from dnemd.nanover.generators import SimpleDNemdGenerator
from dnemd.nanover.nemd_playback import TrajectoryPlayback

# Minimum Working Example
The `TrajectoryPlayback` class is responsible for managing the playback of D-NEMD simulation data using the NanoVer visualiser. To instantiate the `TrajectoryPlayback` class, only one mandatory argument, `universe`, is required. The `universe` entity is an `MDAnalysis.Universe` object that encapsulates the structure and trajectory of the system of interest.

To create a new `universe` instance, the constructor must be supplied with the path (`reference_structure_file_path`) to the PDB file that contains the reference structure of the protein being visualised. Any displacements, provided later, are assumed to be relative to this reference structure.

In [ ]:
reference_structure_file_path = r"PATH\TO\REFERENCE\STRUCTURE\FILE.pdb"
universe = Universe(reference_structure_file_path)

As it stands, the `universe` object is currently incomplete as it does not yet have access to the D-NEMD trajectory. Thus, the next step is to load the D-NEMD displacement data in to a suitable <span style="color: #197fd1; cursor: pointer;" title="Trajectory generators in `MDAnalysis.Universe` instances are crucial for reading molecular dynamics trajectories from diverse file formats. These generators are generally based on the `ReaderBase` class, providing a standard interface for accessing trajectory data. Different trajectory generators are employed by MDAnalysis to accommodate the specific requirements of each file format. Consequently, the introduction of a new file format necessitates the creation of a custom trajectory generator (`SimpleDNemdGenerator`) to ensure proper data handling."><u>trajectory generator</u><sup>🛈</sup></span>, and provide provide that to the `universe` object like so:

In [ ]:
displacement_file_path = rPATH\TO\DISPLACEMENT\DATA\FILE.h5"
trajectory = SimpleDNemdGenerator(displacement_file_path)

universe.trajectory = trajectory

where `displacement_file_path` is a path pointing to an HDF5 file containing the D-NEMD displacement data. This file should be structured according to the schema outlined in the [README](https://github.com/mcsloy/NanoVerNemdParser) file. Note that files written in the now deprecated column-based structured file format can be converted to their HDF5 equivalent as described [here](#Converting-Old-D-NEMD-Data-Files). Now that the `universe` instance has been fully initialised, the `TrajectoryPlayback` class can be instantiated as follows:

In [ ]:
trajectory_player = TrajectoryPlayback(universe)

At this stage, sufficient information has technically been provided to the runner to enable basic playback, which can be initiated by issuing the `trajectory_player.play()` command. However, the visualisation will remain quite limited unless specific rendering settings, as discussed bellow, are configured.

## Visualisation Settings
The `TrajectoryPlayback` class offers numerous configurable settings to control the visual representation of D-NEMD data within NanoVer. However, most of these settings apply specifically to the `"cartoon extended"` renderer. Therefore, users should ensure that the correct renderer is set.
<div style="border-left: 4px solid #f39c12; background-color: #fcf8e3; padding: 10px;">
  <strong>⚠ Warning:</strong> D-NEMD specific visualisation settings will not function unless the <code>"cartoon extended"</code> render is selected!
</div>


In [ ]:
trajectory_player.set_global_renderer("cartoon extended")

All other visualisation settings can be set either via keyword argument to the `TrajectoryPlayback` constructor, or adjusted dynamically at runtime. The `fps` (frames per second) setting controls the playback speed of the trajectory. It determines how many trajectory frames are displayed per second during the visualisation. A higher `fps` results in smoother, faster playback, while a lower `fps` slows down the visualisation, making it easier to observe individual frames. Adjusting this setting allows users to fine-tune the playback speed to match their needs for real-time observation or detailed analysis. By default, the trajectory will loop once it reaches the last frame.

In [ ]:
trajectory_player.fps = 15

The `displacement_scale_factor` setting controls the overall magnitude of the visualised displacements, allowing users to amplify or reduce the apparent motion of residues. By default, displacements are shown at their actual scale, but by increasing the `displacement_scale_factor`, the displacements can be magnified to make subtle movements more noticeable. Conversely, lowering the factor will minimise the visual effect of the displacements. This setting is applied globally, affecting all residues uniformly. It is particularly useful for highlighting small displacements that might otherwise be difficult to see, or for adjusting the visual impact in crowded or complex systems. The `displacement_scale_factor` does not affect the normalised distance metric values used for colouring and scaling (as discussed below), allowing it to be adjusted independently for clearer visualisation.

In [ ]:
trajectory_player.displacement_scale_factor = 1.0

### Per-residue Settings
Regions of high activity can be highlighted by adjusting the visual properties of each residue based on the magnitude of its displacement. Specifically, residues with greater displacement will appear more vibrant in colour and larger in size.

<details>
  <summary>Normalised Displacement Metric</summary>

Displacement magnitudes are not used directly; instead, they are normalised to a metric that spans the domain \[0, 1\], i.e. a *normalised displacement metric*. This normalisation ensures consistent scaling and colouring across different systems, allowing for meaningful visual comparisons to be made regardless of the absolute displacement values. The normalised displacement metric is determined using the following equation:
$$x_{\text{norm}} = \frac{x^p - x_{\text{min}}^p}{x_{\text{max}}^p - x_{\text{min}}^p}$$
where:
- $x_{\text{norm}}$ is the resulting normalised displacement metric.
- $x$ is the magnitude of the residue's displacement.
- $x_{\text{min}}$ is the lower bound (or minimum value for normalisation).
- $x_{\text{max}}$ is the upper bound (or maximum value for normalisation).
- $p$ is the exponent (or power term) which can be used to introduce a degree of non-linearity.

By default, $x_{\text{min}}$ and $x_{\text{max}}$ are set to the minimum and maximum displacement distances in the trajectory, with $p$ set to 1. However, these defaults can be overridden using the `displacement_normalisation_lower_bound`, `displacement_normalisation_upper_bound`, and `displacement_normalisation_exponent` properties of the `TrajectoryPlayback` class.

<div style="border-left: 4px solid #3498db; background-color: #ebf5fb; padding: 10px;">
  <strong>ℹ Notice:</strong> Manually setting or modifying the scaling bounds without good reason is generally discouraged.
</div>

</details>

<div style="margin-top: 20px;"></div>

#### Colour
In order to control the colouring of residues, a Matplotlib [colour map](https://matplotlib.org/stable/users/explain/colors/colormaps.html) must be specified through the `colour_map_name` parameter. This map defines the gradient that will be applied across the normalised displacement metric, which spans the range \[0, 1\]. The colour map associates displacement magnitudes with <span style="color: #197fd1; cursor: pointer;" title="Residues with normalised displacement values at or below the minimum are coloured using the start of the gradient, while those with displacements at or above the maximum take the final colour in the gradient."><u>corresponding colours</u><sup>🛈</sup></span>, with commonly used options such as "viridis" (default) or "plasma" providing visually distinct gradients.

In [ ]:
trajectory_player.colour_map_name = "viridis"

#### Scaling
Residue scaling adjusts the size of each residue based on the magnitude of its displacement, making areas of higher activity visually larger. This effect is controlled by two parameters: `residue_scale_minimum` and `residue_scale_maximum`, which define the scaling range. A residue's scaling factor is determined via the following expression:
$$s_{final} = x_{\text{norm}} (s_{max} - s_{min}) + s_{min} $$
Where $s_{\text{min}}$ and $s_{\text{max}}$ the minimum and maximum reside scale factors respectively, $x_{\text{norm}}$ is the normalised displacement metric of the residue, and $s_{\text{final}}$ is the final scaling factor for the residue. The final factor is then used by the NanoVer client to scale the residue.

This scaling emphasises regions with significant motion, making areas of interest in the molecular structure more visually prominent. By adjusting these parameters, users can control the degree to which displacement is reflected in the residue sizes, enhancing or reducing their visibility as needed.

By default, both the minimum and maximum scale factors are set to 1.0, resulting in no scaling effect. While it is generally recommended to keep the minimum scale factor at 1.0 to avoid shrinking residues, the maximum scale factor can be freely adjusted to amplify the visual impact of displacement scaling as needed.

In [ ]:
trajectory_player.residue_scale_maximum = 4.0

## Execution
By this point, the D-NEMD displacement data and reference structure files have been successfully loaded and parsed into the `TrajectoryPlayback` class. The visualisation settings, such as rendering style, playback speed, displacement scaling, and colour mapping, have been configured. The system is now ready for visualisation playback to be initiated. Playback can be started by calling the `play` method like so:

In [ ]:
trajectory_player.play()

Playback may be temporarily paused using method of the same name:

In [ ]:
trajectory_player.pause()

The pause simulation may then be resumed by invoking the `play` method again.

In [ ]:
trajectory_player.play()

As previously mentioned the visualisation settings may be changed ad-hoc during playback.

In [ ]:
trajectory_player.colour_map_name = "plasma"
trajectory_player.displacement_scale_factor = 4.0
trajectory_player.residue_scale_maximum = 8.0

When finished, users may terminate the server and associated trajectory player via the `close` method:

In [ ]:
trajectory_player.close()

<div style="margin-top: 20px;"></div>

<div style="border-left: 4px solid #ff1f1f; background-color: #ffcfcf; padding: 10px;">
  <strong>⚠ Caution:</strong> It is imperative that the `close` command is issued before closing the script. Failing to do so may result in the background server thread remaining active resulting in undefined behaviour!
</div>

<div style="margin-top: 20px;"></div>

<div style="border-left: 4px solid #f39c12; background-color: #fcf8e3; padding: 10px;">
  <strong>⚠ Warning:</strong> If the `play` method is invoked after closure, a new server will be started. However, due to current limitations, the action of closing and reopening a server clears the renderer. Thus users must re-specify the renderer again; i.e. issue `trajectory_player.set_global_renderer("cartoon extended")`.
</div>

### Miscellaneous

#### Converting Old D-NEMD Data Files
Files written in the now deprecated column-based structured file format following the "`average_xyz_displacement_<TIME>ps`" naming scheme may be converted to their HDF5 equivalent using the `dnemd.parsing.schema._convert_old_files` method. This is demonstrated below:

In [ ]:
from dnemd.parsing.schema import _convert_old_files
directory_path = r"path/to/reference/old_displacement_file_directory"
structure_file_path = r"path/to/reference/structure/file.pdb"
displacement_frames = _convert_old_files(directory_path, structure_file_path)
displacement_frames.write("new_file_name.h5")

Note that if file names do not match the original `"average_xyz_displacement_*ps"` naming convention, then a new regex pattern must be specified via the optional `pattern` argument. This function is a temporary feature branch that is intended to help migrate to the new file format. As such it will be deprecated, and subsequently removed in due course.

#### Advanced Settings

##### `alpha`
The `TrajectoryPlayback` class offers an `alpha` attribute which can be used to override the global transparency value. By default, both colour and transparency are controlled by the supplied colour map. Most matplotlib colour maps, however, are fully opaque. If an alpha value is specified, it will globally override the transparency values set by the colour map. For example, setting:

In [ ]:
trajectory_player.alpha = 0.5

will render the protein at 50 % transparency. This is useful for improving clarity in dense systems or when multiple systems are overlaid on top of one another.  The default value is `None`, meaning the transparency from the colour map will be used. Thus to return control of transparency back to the colour map, simply issue:

In [ ]:
trajectory_player.alpha = None

##### `record_to_file`
NanoVer sessions may be recorded by supplying the `TrajectoryPlayback` class with a `record_to_file` during instantiation like so:

In [ ]:
record_to_file = r"path/to/where/record/should/be/stored/name_without_file_extension"
trajectory_player = TrajectoryPlayback(universe, record_to_file=outp)

This will cause the NanoVer session to be recorded and saved in a pair of files, namely "<FILE_NAME>.traj" and "<FILE_NAME>.state". These can then be played back using the general method used to playback NanoVer-IMD recordings.

#### Twinned Displacement
Note that this is currently an experimental feature and is still a work in progress. Documentation to be added.

In [ ]:
# from dnemd.parsing.pdb import load_pdb_file_as_doubled_mdanalysis_topology
# from dnemd.nanover.generators import DoubledGenerator

# displacement_file_1 = r"path/to/the/displacement/data/file_1.h5"
# displacement_file_2 = r"path/to/the/displacement/data/file_2.h5"

# universe = MDAnalysis.Universe(load_pdb_file_as_doubled_mdanalysis_topology(reference_structure_file_path))

# trajectory = DoubledGenerator([displacement_file_1, displacement_file_2], offset=offset)
# universe.trajectory = trajectory